<a href="https://colab.research.google.com/github/Nafiseh-Izadyar/CSC581B-/blob/main/Finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Introduction to Deep Learning for Image Classification - CSC 581B - A01**
*Nafiseh Izadyar*

In [ ]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

Loading the CIFAR-100 dataset and splitting into train, val, and test

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision.utils import save_image
import time

# Loading dataset

transform = transforms.Compose(
    [
      transforms.RandomHorizontalFlip(0.5),
      transforms.RandomVerticalFlip(0.5),
      transforms.RandomRotation(15),
      transforms.RandomCrop(32,4),
      transforms.ToTensor(),
      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
      transforms.Scale((224,224))
    ]
)


test_transform = transforms.Compose(
    [
      transforms.Resize(32),
      transforms.CenterCrop(32),
      transforms.ToTensor(),
      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
      transforms.Scale((224,224))
    ]
)


# batch Size
batch_size = 128

# loading train and Validation Set
trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
print('Train Size:', len(trainset))

# Validation Set
torch.manual_seed(32)
validation_size = 1000
train_size = len(trainset) - validation_size



train, validation = torch.utils.data.random_split(trainset,[train_size,validation_size]) 

trainloader = torch.utils.data.DataLoader(train, batch_size = batch_size, shuffle = True, num_workers = 4)
validationloader = torch.utils.data.DataLoader(validation, batch_size = batch_size, num_workers = 4)

# loading test set
testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=test_transform)
print('Test Size: ',len(testset))
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers = 4)


print('Classes: ')
print(trainset.classes)

print('Done!')


Checking if the GPU is available 

In [ ]:
if torch.cuda.is_available():
  print('GPU is available')
  device = 'cuda'
else:
  print('No GPU')
  device = 'cpu'

Defining accuracy, plot, and testing functions!

In [ ]:
def accuracy(output,label):
  _, preds = torch.max(output, dim=1)
  return torch.tensor(torch.sum(preds == label).item() / len(preds))

def ploting(printloss,validation_loss,printval,printtrain,learning_rate):
  # plot loss
  plt.plot(printloss,'-o',label='train')
  plt.plot(validation_loss,'-o',label='validation')
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.title('loss')

  plt.figure()

  # plott accuracy on validation set
  plt.plot(printval,'-o',label = 'validation')
  plt.plot(printtrain,'-o',label='train')
  plt.xlabel('Epoch')
  plt.ylabel('Accuracy')
  plt.title('Accuracy of validation set in different epochs')
  plt.legend(loc ="upper left")
  plt.figure()

  # plot learning rate
  plt.plot(learning_rate,'-o')
  plt.xlabel('Epoch')
  plt.ylabel('Learning rate')
  plt.title('Learning Rate Decay')

def testModel(model,test):
  correct = 0
  total = 0
  with torch.no_grad():
    for data in test:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
  print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))


Pretrained models

In [ ]:
import warnings
def select_model(m):
  if m == 'alexnet':
    # AlexNet
    model = models.alexnet(pretrained=True)
    model.classifier[6] = nn.Linear(4096,100)
    model.to(device)
    return model
  elif m == 'resnet18':
    # Resnet18
    model = models.resnet18(pretrained = True)
    model.fc = nn.Linear(512, 100)
    model.to(device)
    return model
  elif m == 'densenet':
    # DenseNet
    model = models.densenet121(pretrained=True)
    model.classifier = nn.Linear(1024, 100)
    model.to(device)
    warnings.warn('Out of Memory Error')
    return model
  elif m == 'googlenet':
    #googlenet
    model = models.googlenet(pretrained=True)
    print(model)
    model.fc = nn.Linear(1024,100)
    model.to(device)
    return model
  elif m == 'inception':
    # Inception
    model = models.inception_v3(pretrained=True)
    model.AuxLogits.fc = nn.Linear(768, 100)
    model.fc = nn.Linear(2048,100)
    model.to(device)
    return model
  elif m == 'squeezenet':
    ##squeezenet
    model = models.squeezenet1_0(pretrained=True)
    model.classifier[1] = nn.Conv2d(512, 100, kernel_size=(1,1), stride=(1,1))
    model.num_classes = 100
    model.to(device)
    return model
  else:
    assert('Model does not exist!')

m = 'alexnet'
model = select_model(m)
print(model)

Defining the optimizer, criteria, and learning rate schedualer

In [ ]:
# Defining the optimizer, criteria, and learning rate scheduler
optimizer = optim.Adam(model.parameters(), lr=0.1,weight_decay=1e-3)
criterion = nn.CrossEntropyLoss()
schedualer = optim.lr_scheduler.MultiStepLR(optimizer,[5,20])
print('Done!')

The validation part has been commented becasue the GPU would ran out of memory, but works if enough memory is available. 

In [ ]:
# Tracking the training time
a = time.time()

# tracking loss
total_loss = []

# tracking train and validation accuracy
val_acc = []
train_acc = []
validation_loss = []

# tracking the learning-rate
learning_rate = []

# train for number of epochs
epochs = 25
for iter in range(epochs):
  
  running_loss = 0.0
  for i, data in enumerate(trainloader,0):
    torch.cuda.empty_cache()
    inputs, labels = data[0].to(device), data[1].to(device)
    del data
    optimizer.zero_grad()
    outputs = model(inputs)

    # calculating the loss
    loss = criterion(outputs,labels)
    loss.backward()
    optimizer.step()

    lr = optimizer.param_groups[0]["lr"]

    running_loss += loss.item()

    if i % 313 == 312:
      print('[%d, %5d] loss: %.3f' % (iter + 1, i + 1, running_loss / 313))
      total_loss.append(running_loss / 313)
      running_loss = 0.0

  
  learning_rate.append(lr)

  # keeping track of train acc
  acc_train = accuracy(outputs,labels)
  train_acc.append(acc_train)
  del inputs, labels

  # Validation
  val_loss = []

  # for j, valdata in enumerate(validationloader,0):
  #   valin, vallabel = valdata[0].to(device), valdata[1].to(device)
  #   del valdata
  #   valout = model(valin) 
  #   val_loss.append(criterion(valout,vallabel))
  #   acc = accuracy(valout,vallabel)
  #   del valin, vallabel, valout
  
  # epoch_loss = torch.stack(val_loss).mean()
  # validation_loss.append(epoch_loss)
  # print('[%d, %5d] val loss: %.3f' % (iter + 1, i + 1, epoch_loss))
  # print('[%d, %5d] val accuracy: %.3f' % (iter + 1, i + 1, acc * 100))
  # val_acc.append(acc)

  schedualer.step()

b = time.time()
print(b-a)

Testing and ploting the results!

In [ ]:
ploting(total_loss,validation_loss,val_acc,train_acc,learning_rate)
testModel(model,testloader)